# Artifical Neural Networks & Deep Learning
# Homework 1 - Image Classification

**Developement Team:**
- Acquati Marco - 10583134 
- Brugali Giorgio - 10794550
- Puoti Francesco - 10595640 


# *1. Data acquisition and augmentation*
> This topic has been issued with annotations as the code flows down, to better clarify the correspondence between the explanations and the code snippets 



# *2. Model overview*


> **2.1. Features' Extraction**

>> The feature extractor is composed by *5 blocks*, each composed by:
  - one convolutional part
  - one  MaxPooling layer. 

>>The convolutional part has a variable number of convolutional layers: 2 Conv2D layers for the first two blocks and 3 Conv2D layers for the remaining ones.
This choice was dictated with the intention of reducing the parameters' number in the network.

>>Our idea was to start with small filters 3x3 because of the need to detail the features' extraction at the beginning. 
Afterwards, instead of increasing the filters' size, to avoid having possible distorted features that could result too effective in the learning process, we decided to augment the number of convolutional layers.

>>Regarding the activation function, we chose the ReLU activation function as it is more suitable for the classification problem.
BatchNormalization has been involved in the features' extraction part in order to both improve the stability of our network and to reduce covariance shift, the latter resulting in improving the training velocity as well. 

>>The pool size has been set to 3x3 with stride 2x2 to favor the overlapping: it has been demonstrated that the overlapping pooling areas reduce the likelihood of the network to overfit. 

> **2.2. BottleNeck Layer**
>> In order to reduce the computational load of the network, 
the number of the features extractor’s output channels is reduced by adding a 1x1 convolutional layer before feeding the output to the classifier.

> **2.3. Classifier**
>> - one flatten layer
>> - two dense hidden layers with 2048 neurons each
>> - the output layer with the SOFTMAX activation function and three classes

>> It's worth highlighting the use of weight initialization (HeNormal distribution), which aims at improving the network speed, avoiding too many zeroes in the kernels at the beginning of the learning process.

>> Moreover, weight decay has been implemented to reduce overfitting in the Dense layers.

>> Both BatchNormalization and ReLU have been used for the same purpose as in the features' extraction part.

> **2.4. Optimizer & LossFunction** 
>> - Adam, with a starting learning rate of 1e-3 and amsgrad = True to have an adaptive learning rate, so as to prevent the network from being stuck on a suboptimal solution.
>> - Loss function : Categorical Crossentropy.

> **2.5. Further information about the implemention process**
>> No EarlyStopping has been used in the final model as, after some trials, such model got stopped even though the learning process would have subsequently led to noteworthy improvements.

>> With regard to the checkpoints, we initially implemented them but, the more the network was becoming complex, the bigger was the space occupied on the HDD.
Therefore, we need to avoid to use checkpoint, otherwise HDD space on kaggle as well as colab would have ran out of memory.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

SEED = 1234
tf.random.set_seed(SEED) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

In [ ]:
import os
import json
import operator

# Since the training images were not divided in subfolders, 
#     we had to manage the data acquisition by means of data frame. 
# First, we acquired the images' paths an we sorted them in order to create 
#     a correspondence between each image and its label stored in the json file.
#-------------------------------------------------------------------------------

X = [] #list of images' paths
for dirname, _, filenames in os.walk('/content/MaskDataset/training'):
    filenames.sort()
    for filename in filenames:
           X.append(os.path.join(dirname, filename))

with open('/content/MaskDataset/train_gt.json') as f:

 data = json.load(f)
 
 data = sorted(data.items(), key=operator.itemgetter(0))

y = [] #list of target labels
for i in range(len(data)):
    y.append(str(data[i][1])) #il dataframe vuole delle string

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# We decided to apply data augmentation on the training set not because of the data shortage
# but in order to make our model more flexibile on recognizing objects in different positions and dimensions.
#------------------------------------------------------------------------------------------------------------
train_data_gen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval= 0,
                                    rescale=1./255)

# No data aumentation has been applied on validation set, since we want to have the images meant for validation 
# similar to the test images to find out the features of our model
#--------------------------------------------------------------------------------------------------------------
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
from sklearn.model_selection import train_test_split

# Data set split in training and validation sets 
# with the latter having a size equal to the 20% of the entire data set.
#-----------------------------------------------------------------------
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

# Generation of training and validation dataframes using pandas' library
#------------------------------------------------------------------------
dataframe_train = pd.DataFrame({"input": X_train, "target": y_train})    
dataframe_valid = pd.DataFrame({"input": X_valid, "target": y_valid})

In [ ]:
# Batch size
bs = 32

# Images spatial shape
img_h = 256
img_w = 256

num_classes = 3
classes = ['0', '1', '2']
clmode = "rgb"


# Creation of the DataFrameIterators yielding tuples of (x, y) 
# where x is a numpy array containing a batch of images with shape (batch_size, *target_size, channels) 
# and y is a numpy array of corresponding labels.
#------------------------------------------------------------------------------------------------------

train_datagen = train_data_gen.flow_from_dataframe(
      dataframe = dataframe_train,
      directory = './',
      x_col = "input",
      y_col = "target",
      target_size = (img_h, img_w),
      color_mode = clmode,
      classes = classes,
      class_mode = "categorical",
      batch_size = bs,
      shuffle = True,
      seed = SEED
)

valid_datagen = valid_data_gen.flow_from_dataframe(
      dataframe = dataframe_valid,
      directory = './',
      x_col = "input",
      y_col = "target",
      target_size = (img_h, img_w),
      color_mode = clmode,
      classes = classes,
      class_mode = "categorical",
      batch_size = bs,
      shuffle = True,
      seed = SEED
)

In [ ]:
# Create Dataset objects
# ----------------------

# Training
#---------
train_dataset = tf.data.Dataset.from_generator(lambda: train_datagen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes])) 
train_dataset = train_dataset.repeat()

# Validation
#-----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_datagen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

In [ ]:
import operator
    
model = tf.keras.Sequential()

depth = 5
start_f = 64
ker_sz = (3,3)
ker_str = (1,1)
pool_sz = (3,3)

initializer = tf.keras.initializers.he_normal(seed=SEED)
regularizer = tf.keras.regularizers.l2(0.001) #weight decay
dpout = 0.5

convNumb = 2

# Features extraction
# -------------------
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]
        
    if i == 2:
        convNumb += 1 #increasing number of Conv2D layer in a block
        
    for j in range(convNumb):
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                 kernel_size=ker_sz,
                                 strides=ker_str,
                                 padding='same',
                                 input_shape=input_shape))        
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_sz, strides = 2))
    if start_f < 512:
        start_f *= 2

#Bottle neck layer
#-----------------
model.add(tf.keras.layers.Conv2D(filters=start_f/2, 
                              kernel_size=(1,1),
                              strides=ker_str,
                              padding='same',
                              input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())     

#Classifier
#----------
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=2048, 
                                kernel_regularizer = regularizer,
                                kernel_initializer = initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU()) 
model.add(tf.keras.layers.Dropout(dpout))
model.add(tf.keras.layers.Dense(units=2048,
                                kernel_regularizer=regularizer, 
                                kernel_initializer = initializer))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(x=train_dataset,
          epochs=150,
          steps_per_epoch=len(train_datagen),
          validation_data=valid_dataset,
          validation_steps=len(valid_datagen)
         )

In [ ]:
#--------------Saving the model---------------
#---------------------------------------------

from datetime import datetime

savedir ='/content/drive/MyDrive/savedModels'

if not os.path.exists(savedir):
  os.makedirs(savedir) 
  
savePath =  os.path.join(savedir, 'model' + datetime.now().strftime('%b%d_%H-%M-%S')+'.h5')
model.save(savePath)

In [ ]:
clmode = "rgb"
source = '/content/MaskDataset/'

test_data_gen = ImageDataGenerator(rescale = 1./255)

test_datagen = test_data_gen.flow_from_directory(
    source,
    target_size = (256, 256),
    color_mode = clmode,
    classes =  ["test"],
    class_mode = "categorical",
    batch_size = 1,
    shuffle = False
)
test_datagen.reset()

In [ ]:
predictions = model.predict_generator(test_datagen, len(test_datagen), verbose = 1)
result = {}

In [ ]:
import ntpath

images = test_datagen.filenames
i = 0

for p in predictions:
    prediction = np.argmax(p)
    image_name = ntpath.basename(images[i])
    result[image_name] = str(prediction)
    i = i+1


In [ ]:
from datetime import datetime

def create_csv(results):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join('/content', csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
    return csv_fname 
    # create_csv returns the path of the csv file, which is currently in the session folder, since google Drive rises an exception if we try to write a file              # directly in it. therefore, we first create the file and then, as it follows in the next cell, we move the csv file from the session folder to the drive

In [ ]:
file_to_move = create_csv(result)

import shutil
shutil.move(file_to_move, '/content/drive/MyDrive/') # moving the csv file